# Criando a estrutura do código

#### Importando as bibliotecas

In [1]:
import pandas as pd
import pickle
import json
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer, StandardScaler

#### Ajustando os paths

In [2]:
from pathlib import Path
import os

data_path = str(Path(os.getcwd()).parent / "data")
artifacts_path = str(Path(os.getcwd()).parent / "artifacts")
logs_path = str(Path(os.getcwd()).parent / "logs")

#### Função para registrar os logs

In [3]:
import logging

# Configuração do logger
LOG_DUMP_PATH = f'{logs_path}/nt_failure.log'

logging.basicConfig(
    filename=LOG_DUMP_PATH,
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s',
)

def _log_failure(e):
    logging.error(e)

#### Carregamento dos dados

In [4]:

def load_data(file_path: str) -> pd.DataFrame:
    """Carrega o dataset em formato Parquet."""
    try:
        return pd.read_parquet(file_path)
    except Exception as e:
        _log_failure(e)

#### Carregamento do modelo

In [5]:
def load_model(file_path: str):
    """Carrega o modelo treinado salvo em pickle."""
    try:
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    except Exception as e:
        _log_failure(e)

#### Carregamento da pipeline

In [6]:
def load_pipeline(file_path: str) -> Pipeline:
    try:
        # Carrega o arquivo JSONC
        with open(file_path, 'r') as f:
            str_json = '\n'.join(f.readlines()[3:])
            pipeline_config = json.loads(str_json)

        # Configura as etapas do pipeline
        steps = []

        if "reduce_dim" in pipeline_config["steps"]:
            steps.append(("reduce_dim", PolynomialFeatures(**pipeline_config["steps"]["reduce_dim"]["PolynomialFeatures"])))

        if "qtransf" in pipeline_config["steps"]:
            steps.append(("qtransf", QuantileTransformer(**pipeline_config["steps"]["qtransf"]["QuantileTransformer"])))

        if "poly_feature" in pipeline_config["steps"]:
            steps.append(("poly_feature", PolynomialFeatures(**pipeline_config["steps"]["poly_feature"]["PolynomialFeatures"])))

        if "stdscaler" in pipeline_config["steps"]:
            steps.append(("stdscaler", StandardScaler(**pipeline_config["steps"]["stdscaler"]["StandardScaler"])))

        # Retorna o pipeline
        return Pipeline(steps)
    except Exception as e:
        _log_failure(e)

#### Função de Score

In [7]:
def score(data_path: str, model_path: str, pipeline_path: str):
    try:
        # Carregar o modelo, os dados e o pipeline
        data = load_data(data_path)
        model = load_model(model_path)
        pipeline = load_pipeline(pipeline_path)

        # Seleciona as colunas de interesse
        data = data[['vibration_x', 'vibration_y', 'vibration_z']]

        # Ajusta o pipeline com os dados
        pipeline.fit(data)
        transformed_data = pipeline.transform(data)

        if not len(transformed_data):
            raise RuntimeError('No data to score')
        if not hasattr(model, 'predict'):
            raise Exception('Model does not have a predict function')

        # Retorna a previsão do modelo
        return model.predict(transformed_data)

    except Exception as e:
        print(e)
        _log_failure(e)


#### Execução da pipe.

In [8]:
# Caminhos dos arquivos
data_path = f'{data_path}/dataset.parquet'
model_path = f'{artifacts_path}/model.pkl'
pipeline_path = f'{artifacts_path}/pipeline.jsonc'

# Executar a função de scoring
result = score(data_path, model_path, pipeline_path)
print(result)

[-1 -1  1  1 -1 -1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1
 -1  1 -1 -1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1 -1 -1  1  1  1 -1  1  1  1  1 -1  1 -1  1  1  1
 -1 -1 -1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1 -1  1 -1  1 -1 -1]


c:\Users\ernan\anaconda3\envs\venv-shape-challenge\lib\site-packages\sklearn\preprocessing\_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (295). n_quantiles is set to n_samples.
  warnings.warn(
